In [170]:
import pandas as pd

In [171]:
transactions=data = pd.read_csv('train.csv')
# transactions[transactions['amount_flag']==True][transactions['customer_id']=="a1e8e883-8f1a-4a02-91c8-c9157fcebf67"]
# customer_concentration = transactions.groupby(['merchant_id', 'customer_id']).size().reset_index(name='txn_count')
# print(customer_concentration)  # Check customer-level transaction counts
# max_txns_per_customer = customer_concentration.groupby('merchant_id')['txn_count'].max().reset_index(name='max_txns_per_customer')
# print(max_txns_per_customer.head())  # Check if maximum counts are as expected
# total_txns_per_merchant = transactions.groupby('merchant_id')['transaction_id'].count().reset_index(name='total_txns')
# print(total_txns_per_merchant.head())  # Check total transactions per merchant
transactions

,transaction_id,merchant_id,timestamp,amount,customer_id,device_id,customer_location,payment_method,product_category,platform,velocity_flag,amount_flag,time_flag,label
0,4b070b85-1f84-4cb8-b113-ceefa107649e,M1025,2024-11-15T01:19:34,215.49,174d4f19-c408-40b0-a605-c6dbaf661a1b,1c58be65-c7aa-47bb-8c0a-f67e4147c6ed,South Aaronburgh,Net Banking,Clothing,Mobile,False,False,False,0
1,8a44ce71-2dcd-40be-8c05-594af0dadfbf,M1040,2024-11-05T23:01:43,101.10,77ae89f3-1e6d-43a7-bbea-b605f27b2b0e,c116f041-64a9-4579-bab0-700a9c33c6d1,Heatherchester,Credit Card,Electronics,Mobile,False,False,False,0
2,6e32d150-de51-42e4-b6a8-790050c55fe2,M1073,2024-11-14T20:12:19,411.83,88a62f54-1f95-4913-a12d-085c1412274f,dec389b4-875a-4b0f-b199-a3457d5f8b0d,Michaelburgh,UPI,Electronics,Mobile,False,False,False,0
3,2ef25388-68e9-4fc3-ac02-3da9e61e5218,M1062,2024-11-28T23:05:43,363.12,dad807a7-de71-4542-bfa6-b5c1c86eccc6,189c9cf4-fd57-4a0c-aeb9-c02ee2b1adbc,South Paul,Net Banking,Electronics,Mobile,False,False,True,1
4,124ed179-74c0-431f-819e-82b3bd2f8de2,M1093,2024-11-06T19:18:51,127.04,c4466c5e-ee96-4f02-919d-98c28ea3c405,cf76f8f6-d3ff-4a2b-8b15-1b347f3e7490,Lake Jeremy,Net Banking,Groceries,Mobile,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,669847af-6a1c-4451-aea7-a3b44e287e76,M1045,2024-11-28T23:27:19,199.09,545b75a8-c1e6-460b-af67-9edf70386375,0d480170-0628-4965-99b2-da1f8fa52a71,Reginahaven,Credit Card,Electronics,Web,False,False,True,1
9996,16cfcf5d-dd54-4790-be2d-f5266ed777ff,M1029,2024-11-02T12:01:12,293.91,9ebdf1de-b926-4030-829a-cb2ba14c81a9,5c896174-af0d-404e-ba16-954bb75a7ee1,Cartershire,Debit Card,Electronics,Web,False,False,False,0
9997,e6a65211-5e66-4c26-88c3-67195f66d25c,M1035,2024-10-31T15:06:25,188.88,ee08ff3b-97a1-4855-acf3-5e32fecccecf,aace00a2-e4d2-4c15-ac05-672133cc91b0,Lake Darlene,Credit Card,Groceries,Web,False,False,False,0
9998,5d052618-12ba-49e3-917b-1b40dd0bc5b9,M1036,2024-11-04T23:02:16,68.68,48a5d616-4fa5-46b6-a123-4e514acc649d,3e54d84f-844a-4507-8dc5-63992bbea745,Pennyburgh,Net Banking,Clothing,Mobile,False,False,False,0


In [172]:
# Convert timestamp to datetime for processing
transactions['timestamp'] = pd.to_datetime(transactions['timestamp'])

# Calculate the number of transactions per day for each merchant
transactions['date'] = transactions['timestamp'].dt.date
velocity_metrics = transactions.groupby('merchant_id').apply(
    lambda group: group['transaction_id'].count() / group['date'].nunique()
).reset_index(name='transactions_per_day')

# Merge the velocity metrics back to the transactions DataFrame
transactions = transactions.merge(velocity_metrics, on='merchant_id', how='left')

In [173]:
# Extract the hour from the timestamp
transactions['hour'] = transactions['timestamp'].dt.hour

# Add a late-night flag for transactions between 11 PM and 4 AM
transactions['late_night'] = transactions['hour'].apply(
    lambda x: 1 if 23 <= x or x < 4 else 0
)

In [174]:
# Calculate amount statistics per merchant
amount_stats = transactions.groupby('merchant_id')['amount'].agg(
    ['mean', 'median', 'std', 'max', 'min']
).reset_index()

# Rename columns for better readability
amount_stats.columns = ['merchant_id', 'mean_amount', 'median_amount', 'std_amount', 'max_amount', 'min_amount']

# Merge amount statistics back into the transactions DataFrame
transactions = transactions.merge(amount_stats, on='merchant_id', how='left')


In [175]:
# Count transactions per customer for each merchant
customer_concentration = transactions.groupby(['merchant_id', 'customer_id']).size().reset_index(name='txn_count')

# Find the maximum transaction count per customer for each merchant
customer_concentration = customer_concentration.groupby('merchant_id')['txn_count'].max().reset_index(name='max_txns_per_customer')

# Calculate concentration ratio
total_txns_per_merchant = transactions.groupby('merchant_id')['transaction_id'].count().reset_index(name='total_txns')
customer_concentration = customer_concentration.merge(total_txns_per_merchant, on='merchant_id')
customer_concentration['concentration_ratio'] = customer_concentration['max_txns_per_customer'] / customer_concentration['total_txns']

# Merge the concentration ratio back into the transactions DataFrame
transactions = transactions.merge(customer_concentration[['merchant_id', 'concentration_ratio']], on='merchant_id', how='left')

In [176]:
from sklearn.preprocessing import MinMaxScaler

# Select the features to normalize
feature_columns = ['transactions_per_day', 'mean_amount', 'std_amount', 'concentration_ratio']

# Initialize a MinMaxScaler
scaler = MinMaxScaler()

# Normalize the selected features

transactions[feature_columns] = scaler.fit_transform(transactions[feature_columns])
transactions
transactions.to_csv("normalise_train.csv",index=False)